# Download data 


Downloading the ECG 2020 datasets and copy to google drive

In [ ]:
# !wget -r -N -c -np https://physionet.org/files/ptb-xl/1.0.1/ 

In [ ]:
# %cp -r -np /content/physionet.org/files/ptb-xl /content/gdrive/My\ Drive/ECG2020new

In [ ]:
# #%cd /content/physionet.org/files/ptb-xl/
# %cd 20000/

In [ ]:
# %ls

In [ ]:
# %pwd

In [ ]:
#%cd physionet.org/files/ptb-xl/1.0.1/

Load GPU

In [ ]:
# %tensorflow_version 2.x
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# Link to Google Drive Import library

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

Install library

In [ ]:
# !pip install wfdb

Import library

In [1]:
import numpy as np
import pandas as pd
import wfdb
import matplotlib.pyplot as plt
import ast
import os
import tensorflow as tf
#from keras.utils import to_categorical

#import tensorflow_datasets as tfds
import datetime

from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow_addons.optimizers import CyclicalLearningRate
#from keras.layers import LSTM

import pywt

# read tfrecord

In [2]:
def _parse_function(tf_data):
  features = {"data": tf.io.FixedLenFeature((), tf.string),
              "label": tf.io.FixedLenFeature((), tf.string)}
  parsed_features = tf.io.parse_single_example(tf_data, features)
  data = tf.io.parse_tensor(parsed_features['data'], out_type = tf.float32)
  labels = tf.io.parse_tensor(parsed_features['label'], out_type = tf.float32)
  return data, labels

def read_from_path(tf_record_path):
    dataset = tf.data.TFRecordDataset(tf_record_path)
    dataset = dataset.map(_parse_function)
    return dataset

In [3]:
tf_record_train_path = 'C:\\Users\\jerry\\Downloads\\tfrecord\\train.tfrecords'
tf_record_test_path = 'C:\\Users\\jerry\\Downloads\\tfrecord\\test.tfrecords'
tf_record_val_path = 'C:\\Users\\jerry\\Downloads\\tfrecord\\val.tfrecords'
#TODO LIST:
# read train dataset here
train_dataset = read_from_path(tf_record_train_path)
# read test dataset here
test_dataset = read_from_path(tf_record_test_path)
# read validation dataset here
val_dataset = read_from_path(tf_record_val_path)

In [105]:
for i in dataset.take(3):
    print(i)

(<tf.Tensor: shape=(5000, 12), dtype=float32, numpy=
array([[-0.02815411,  0.05928221,  0.08811308, ..., -0.15034847,
         0.10512699, -0.02803453],
       [-0.02813395,  0.05913024,  0.0880912 , ..., -0.1507113 ,
         0.10493328, -0.02797886],
       [-0.02815004,  0.0590063 ,  0.08804197, ..., -0.15101314,
         0.10476465, -0.0278557 ],
       ...,
       [-0.08832211,  0.01723753,  0.1047522 , ...,  0.49859625,
         0.10776633, -0.24827701],
       [-0.08821018,  0.01708768,  0.10444412, ...,  0.4985945 ,
         0.10800342, -0.24830152],
       [-0.08818521,  0.01692302,  0.10416713, ...,  0.4986349 ,
         0.10823737, -0.24836968]], dtype=float32)>, <tf.Tensor: shape=(2,), dtype=float32, numpy=array([0., 1.], dtype=float32)>)
(<tf.Tensor: shape=(5000, 12), dtype=float32, numpy=
array([[ 0.18461645,  0.02534221, -0.15855102, ...,  0.46917713,
         0.06536251, -0.04939149],
       [ 0.18551959,  0.02606068, -0.15855682, ...,  0.46896774,
         0.06553638, 

# Get the label from csv, set up the basic raw data, get the features

In [3]:
#get path
path='C:\\Users\\jerry\\Downloads\\WFDB\\'
# path = '/content/gdrive/.shortcut-targets-by-id/1ca8px2VUzY6uBCgo13kYoW0_JuynXfG7/1.0.1/'
tf_save_path = 'C:\\Users\\jerry\\Downloads\\tfrecord\\'

In [4]:
def get_label_info(path):
  def add_path(str2):
      return path + "HR"+str2.split("_")[0]
  def aggregate_diagnostic(y_dic):
      tmp = []
      for key in y_dic.keys():
          if key in agg_df.index:
              tmp.append(agg_df.loc[key].diagnostic_class)
      if len(tmp) == 0:
        return "NL"
      return list(set(tmp))[0]
  Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
  Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

      # Load scp_statements.csv for diagnostic aggregation
  agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
  agg_df = agg_df[agg_df.diagnostic == 1]

      # Apply diagnostic superclass
      #target label of the classification
  Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
  Y['new_filename'] = Y.filename_hr.map(add_path)
  Y['strat_fold'] = Y.strat_fold
  return Y[["new_filename","diagnostic_superclass","age","sex","strat_fold"]]

In [5]:
y = get_label_info(path)
y.head()

,new_filename,diagnostic_superclass,age,sex,strat_fold
ecg_id,,,,,
21837,C:\Users\jerry\Downloads\WFDB\HR21837,NORM,68.0,0,9
21836,C:\Users\jerry\Downloads\WFDB\HR21836,NORM,64.0,1,8
21835,C:\Users\jerry\Downloads\WFDB\HR21835,STTC,59.0,1,2
21834,C:\Users\jerry\Downloads\WFDB\HR21834,NORM,93.0,0,4
21833,C:\Users\jerry\Downloads\WFDB\HR21833,STTC,67.0,1,7


In [6]:
y = y[y.diagnostic_superclass !="NL"]
y.diagnostic_superclass.value_counts()

NORM    9111
CD      4907
STTC    3356
MI      3323
HYP      731
Name: diagnostic_superclass, dtype: int64

In [7]:
y = y.dropna(how='any',axis=0) 
y.diagnostic_superclass.value_counts()

NORM    9105
CD      4861
STTC    3342
MI      3307
HYP      728
Name: diagnostic_superclass, dtype: int64

Convert the label into numercial data

In [8]:
numeric_label = {"diagnostic_superclass":{"NORM":0,"CD":1,"MI":1,"STTC":1,"HYP":1}}

In [9]:
y = y.replace(numeric_label)
y.diagnostic_superclass.value_counts()

1    12238
0     9105
Name: diagnostic_superclass, dtype: int64

In [10]:
y.new_filename.isnull().value_counts()

False    21343
Name: new_filename, dtype: int64

In [11]:
y.diagnostic_superclass.isnull().value_counts()

False    21343
Name: diagnostic_superclass, dtype: int64

In [12]:
# Split data into train and test
test_fold = 10
val_fold = 9
train_fold = [1,2,3,4,5,6,7,8]

# Train
train_df = y[(y.strat_fold.isin(train_fold))]
# Test
test_df = y[(y.strat_fold == test_fold)]
#test_df = df_new.loc[df_new["strat_fold"]. isin(test_fold)]
val_df = y[(y.strat_fold == val_fold) ]


# #split the train test dataset
# train_df, test_df = train_test_split(df_new, test_size = 0.2)
# #split the train validation dataset
# train_df, val_df = train_test_split(train_df, test_size = 0.2)

In [13]:
#shuffle the dataframe and re_index the dataset
def shuffle_dataset(df):
  df = df.sample(frac=1)
  df = df.reset_index()
  return df

In [14]:
train_df = shuffle_dataset(train_df)
test_df = shuffle_dataset(test_df)
val_df = shuffle_dataset(val_df)

In [15]:
train_df

,ecg_id,new_filename,diagnostic_superclass,age,sex,strat_fold
0,1223,C:\Users\jerry\Downloads\WFDB\HR01223,0,87.0,0,4
1,12275,C:\Users\jerry\Downloads\WFDB\HR12275,0,49.0,0,5
2,181,C:\Users\jerry\Downloads\WFDB\HR00181,1,73.0,0,4
3,14186,C:\Users\jerry\Downloads\WFDB\HR14186,0,51.0,0,3
4,12861,C:\Users\jerry\Downloads\WFDB\HR12861,0,49.0,0,2
...,...,...,...,...,...,...
17045,1495,C:\Users\jerry\Downloads\WFDB\HR01495,0,60.0,1,3
17046,15678,C:\Users\jerry\Downloads\WFDB\HR15678,0,65.0,1,1
17047,17972,C:\Users\jerry\Downloads\WFDB\HR17972,0,62.0,1,8
17048,5948,C:\Users\jerry\Downloads\WFDB\HR05948,0,47.0,1,2


In [16]:
train_df.diagnostic_superclass.value_counts()

1    9782
0    7268
Name: diagnostic_superclass, dtype: int64

Resampling the data label

In [17]:
def resample_df_two_class(df_majority,df_minority,n_samples ):
    #up sampling the miniority class of dataset
  df_minority_upsampled = resample(df_minority, 
                                  replace=True,     # sample with replacement
                                  n_samples= n_samples,    # to match majority class
                                  random_state=123) # reproducible results
  df_new = pd.concat([df_majority, df_minority_upsampled], ignore_index= True)
  #df_new.diagnostic_superclass.value_counts()
  return df_new

In [18]:
df_majority = train_df[train_df.diagnostic_superclass==1]
df_minority = train_df[train_df.diagnostic_superclass==0]
n_samples = len(df_majority)
train_df = resample_df_two_class(df_majority,df_minority,n_samples)
train_df.diagnostic_superclass.value_counts()

1    9782
0    9782
Name: diagnostic_superclass, dtype: int64

In [19]:
train_df.strat_fold.value_counts()

3    2525
8    2475
1    2452
5    2450
2    2443
7    2427
4    2410
6    2382
Name: strat_fold, dtype: int64

In [20]:
test_df.diagnostic_superclass.value_counts()

1    1231
0     917
Name: diagnostic_superclass, dtype: int64

In [21]:
val_df.diagnostic_superclass.value_counts()

1    1225
0     920
Name: diagnostic_superclass, dtype: int64

In [22]:
print("train dataset information:")
print(train_df.info())
print()
print("test dataset information:")
print(test_df.info())
print()
print("validation dataset information:")
print(val_df.info())

train dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19564 entries, 0 to 19563
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ecg_id                 19564 non-null  int64  
 1   new_filename           19564 non-null  object 
 2   diagnostic_superclass  19564 non-null  int64  
 3   age                    19564 non-null  float64
 4   sex                    19564 non-null  int64  
 5   strat_fold             19564 non-null  int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 917.2+ KB
None

test dataset information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2148 entries, 0 to 2147
Data columns (total 6 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   ecg_id                 2148 non-null   int64  
 1   new_filename           2148 non-null   object 
 2   diagnostic_superclass  2148 non-nu

# Read data into Tensorflow data pipeline

In [34]:
def denoise_tfrecord(data,file_path):
    try:
     # Generating numpy array 
      data_array = data.T
      #print(data_array.shape)
      for i, sig_array in enumerate(data_array):
        #print("loop start!")
        sig_array2 = denoise(sig_array,i)
        #print("loop shape:",sig_array2.shape)
        data_array[i] = sig_array2
      data2 = data_array.T
#       if data_array.T.shape[0] != 5000 or data_array.T.shape[1] != 12:
# #       if data_array    
#           print("data_array shape:",data_array.T.shape)
#           print("data shape:",data2.shape)
      return data2        
    except:
      print('Denoise error!')
      print(file_path)
      #print(data.numpy())
      return data

TensorFlow Encode and Decode base Function

In [98]:
def _bytes_feature(value):
    """Returns a bytes_list from a string / byte."""
    # If the value is an eager tensor BytesList won't unpack a string from an EagerTensor.
    if isinstance(value, type(tf.constant(0))):
        value = value.numpy() 
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
    """Returns a float_list from a float / double."""
    return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
    """Returns an int64_list from a bool / enum / int / uint."""
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def save_tfrecords(data, labels, desfile):
    with tf.io.TFRecordWriter(desfile) as writer:
        
       for i in range(len(data)):
           features = tf.train.Features(
                    feature = {
                        "data": _bytes_feature(tf.io.serialize_tensor(data[i])),
                        "label":_bytes_feature(tf.io.serialize_tensor(labels[i])),
    #                     "sex":_int64_feature(sex),
    #                     "age":_float_feature(age)
                    }
                )
           example = tf.train.Example(features = features)
           serialized = example.SerializeToString()
           writer.write(serialized)

In [ ]:
# def convert_data_file_to_tfdata_allpoints(data_path, label_1):
  
#   path_names=os.listdir(data_path)
#  # for name in path_names:
#   #  if not os.path.isdir(name):
# #      path_names.remove(name) 
  
#   for patient_path in path_names:
#       PATH=os.path.join(data_path,patient_path)   #path
#       #if not os.path.isfile(PATH):
#       FILES=os.listdir(PATH)
#          #print(FILES) 
#         # data_files=[]             
#       for file in FILES:
#         if '.hea' in file:
#               #print(file)
#               # data_files.append(file)
#           records = []      
#           labels = []    
#           record_path=os.path.join(PATH,file.split('.')[0])
#           record= wfdb.rdrecord(record_path)
#           label = record.comments[4].split(':')[1]

#           if  label == label_1:
#             label = 1
#           else:
#             label = 0
#           #print(label,file)  
#           sig_data = record.p_signal.T
#           # for single_sig in sig_data:
#           #   sigle_sig = denoise(single_sig)
#           tf_path = os.path.join(data_path+'tfrecord_all',record.record_name+'.tfrecords')
#           print(tf_path)
#           save_tfrecords(sig_data, label, tf_path)
#           #tf.data.experimental.save(dataset, tf_path) 

# #convert_data_file_to_tfdata_allpoints(data_path,' Myocardial infarction')

**here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use but we use 500 hz dataset in our project here**

In [23]:
# here can load the raw data directly to memory, sampling_rate 100 means the 100hz dataset, physinet provide 100hz dataset for user easier to use
# but we use 500 hz dataset in our project here

# def load_raw_data(path):
#     print("start")
#     sampling_rate = 100
#     def aggregate_diagnostic(y_dic):
#       tmp = []
#       for key in y_dic.keys():
#           if key in agg_df.index:
#               tmp.append(agg_df.loc[key].diagnostic_class)
#       return list(set(tmp))
#     # load and convert annotation data
#     Y = pd.read_csv(path+'ptbxl_database.csv', index_col='ecg_id')
#     Y.scp_codes = Y.scp_codes.apply(lambda x: ast.literal_eval(x))

#     # Load scp_statements.csv for diagnostic aggregation
#     agg_df = pd.read_csv(path+'scp_statements.csv', index_col=0)
#     agg_df = agg_df[agg_df.diagnostic == 1]

#     # Apply diagnostic superclass
#     #target label of the classification
#     Y['diagnostic_superclass'] = Y.scp_codes.apply(aggregate_diagnostic)
#     print(len(Y.diagnostic_superclass))
#     #Y.reset_index()
#     # 100 HZ dataset
#     # if sampling_rate == 100:
#     #     data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_lr]
#     #     #print(data)
#     # else:
#     # 500 HZ array dataset
#     for i in range(1,len(Y)+1):
#       f = Y.filename_hr[i]
#       # print("file name:")
#       # print(f)
#       sex = Y.sex[i]
#       # print("sex:")
#       # print(sex)
#       age = Y.age[i]
#       # print("age:")
#       # print(age)
#       data =wfdb.rdrecord(path+f).p_signal.T
#       # print("data:"+str(i))
#       # print(data)
#       tf_path = os.path.join(tf_save_path,f+'.tfrecords')
#       # print("tf_path:")
#       # print(tf_path)
#       label = Y.diagnostic_superclass[i]
#       # print("label:")
#       # print(label)
#       save_tfrecords(data,label,sex,age,tf_path)
#     #data = [wfdb.rdrecord(path+f).p_signal.T for f in Y.filename_hr]
#     #data = np.array([signal for signal, meta in data])
#     print('finish')
#     return data, Y.diagnostic_superclass

In [26]:
#tensor flow function
# def load_data(df,batch_size,i):
#   #print(file_path)
#   #df.reindex()
#   end_index = (i+1)*batch_size
#   if end_index > df.index[-1]:
#     end_index = df.index[-1]
#   start_index = i*batch_size
#   #print('start:',start_index)
#   #print('end:',end_index)
#   path_list = df['new_filename'][start_index:end_index]
#   #print(i)
#   #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
#   #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
#   #label_one_hot = tf.one_hot(df['diagnostic_superclass'][start_index: end_index],5)
#   #label_tensor = tf.constant(label_one_hot,dtype="float32")
#   #file = file_path.numpy().decode('UTF-8')
#   #print(file)
#   data_list = []
#   path_list=path_list.reset_index()
#   for j in range(len(path_list)):
#     #print(j)
#     #print(len(path_list))
#     #print(path_list.index[j])
#     #print(path_list[1])
#     file_path = path_list.new_filename[j]
    
#     data = wfdb.rdrecord(file_path).p_signal.T.astype("float16")
#     if data is None:
#         print(file_path,"error!!")
#     data_list.append(data)
#   data_array = np.array(data_list)
#   #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_tensor))
#   #set tensorflow feature
#   label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],2)
#   dataset = tf.data.Dataset.from_tensor_slices((data_array, label_tensor))
#   return dataset

def load_data(df,batch_size,i,name):
  #print(file_path)
  #df.reindex()
  end_index = (i+1)*batch_size
  if end_index > df.index[-1]:
    end_index = df.index[-1]
  start_index = i*batch_size
  #print('start:',start_index)
  #print('end:',end_index)
  path_list = df['new_filename'][start_index:end_index]
  #print(i)
  #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
  #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
  labels_one_hot = tf.one_hot(df['diagnostic_superclass'][start_index: end_index],2)
  data_list = []
  path_list=path_list.reset_index()
  for j in range(len(path_list)):
    #print(j)
    #print(len(path_list))
    #print(path_list.index[j])
    #print(path_list[1])
    file_path = path_list.new_filename[j]
    
    data = denoise_tfrecord(wfdb.rdrecord(file_path).p_signal.astype("float16"))
    if data is None:
        print(file_path,"error!!")
    data_list.append(data)
  data_array = np.array(data_list)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_one_hot))
  #set tensorflow feature
  #label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],2)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array, labels_one_hot))
  tf_path = os.path.join(tf_save_path+name,'.tfrecords')
  save_tfrecords(data_array, labels_one_hot, tf_path)

# return dataset

def one_hot_encode(signal,label):
    if label == 0:
      label = np.array([1.0,.0]).astype("float32")
    else:
      label = np.array([.0,1.0]).astype("float32")
    return signal , label

In [107]:
def load_data_all(df,name,label_num):
  path_list = df['new_filename']
  #print(i)
  #age_tensor = tf.constant(df['age'][start_index: end_index],dtype="int8")
  #sex_tensor =tf.constant(df['sex'][start_index: end_index],dtype="int8")
  labels_one_hot = tf.one_hot(df['diagnostic_superclass'],label_num)
  data_list = []
  path_list=path_list.reset_index()
  tf_path = os.path.join(tf_save_path,name+'.tfrecords')
  for j in range(len(path_list)):
    file_path = path_list.new_filename[j]
    
    data = wfdb.rdrecord(file_path).p_signal
    data = denoise_tfrecord(data,file_path).astype("float32")
    #print(data.shape)
    
    if data is None:
        print(file_path,"error!!")
    #save_tfrecords(data, labels_one_hot[j], tf_path)
    data_list.append(data)
    
  data_array = np.array(data_list)
  save_tfrecords(data_array, labels_one_hot, tf_path)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array,age_tensor,sex_tensor,label_one_hot))
  #set tensorflow feature
  #label_tensor = tf.constant(train_df['diagnostic_superclass'][start_index: end_index],2)
  #dataset = tf.data.Dataset.from_tensor_slices((data_array, labels_one_hot))

In [110]:
load_data_all(train_df,"train",2)

D:\Anaconda\lib\site-packages\pywt\_thresholding.py:23: RuntimeWarning: invalid value encountered in true_divide
  thresholded = (1 - value/magnitude)


In [60]:
train_df.head()

,ecg_id,new_filename,diagnostic_superclass,age,sex,strat_fold
0,16210,C:\Users\jerry\Downloads\WFDB\HR16210,1,77.0,1,6
1,4590,C:\Users\jerry\Downloads\WFDB\HR04590,1,71.0,1,4
2,11389,C:\Users\jerry\Downloads\WFDB\HR11389,1,54.0,0,6
3,17884,C:\Users\jerry\Downloads\WFDB\HR17884,1,33.0,1,2
4,5912,C:\Users\jerry\Downloads\WFDB\HR05912,1,72.0,1,3


In [25]:
# def check_data(df,batch_size,i):
#   #print(file_path)
#   #df.reindex()
#   print(i)
#   end_index = (i+1)*batch_size
#   if end_index > df.index[-1]:
#     end_index = df.index[-1]
#   start_index = i*batch_size
#   #print('start:',start_index)
#   #print('end:',end_index)
#   path_list = df['new_filename'][start_index:end_index]

#   labels_one_hot = tf.one_hot(df['diagnostic_superclass'][start_index: end_index],2)

#   data_list = []
#   name_list = []
#   path_list=path_list.reset_index()
#   for j in range(len(path_list)):
#     file_path = path_list.new_filename[j]
#     record = wfdb.rdrecord(file_path)
#     data = record.p_signal.astype("float16").T
#     record_name = file_path.split("/")[-1]
#     #print(record_name)
#     for i,signal_data in enumerate(data):
#         check_data = denoise(signal_data,record_name,i)
        
        
    
#     #show(data,labels_one_hot[j].numpy())
#     if data is None:
#         print(file_path,"error!!")
#     data_list.append(data)
#     name_list.append(record_name)
#   data_array = np.array(data_list)
#   name_array = np.array(name_list)
#   dataset = tf.data.Dataset.from_tensor_slices((data_array, labels_one_hot,name_array))
#   return dataset

In [26]:
# k = int(len(df_new)/2048)+1
# for i in range(0,k):
#     print(i)
#     show_dataset_bactched(get_dataset(df_new,2048,i))

# data preprocessing 

In [31]:
# def denoise(sig_data):
#     # wavedec
#     coeffs = pywt.wavedec(data=sig_data, wavelet='db8', level=9)
#     cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
#     #print('here')
#     # threshold
#     threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
#     cD1.fill(0)
#     cD2.fill(0)
#     for i in range(1, len(coeffs) - 2):
#        coeffs[i] = pywt.threshold(coeffs[i], threshold)

#     rdata = pywt.waverec(coeffs=coeffs, wavelet='db8')
#     #print(rdata)
#     return rdata
def denoise(sig_data,sig_num):
    # wavedec
#     print(record_name)
        #print("denoise data:",sig_data)
        #print(sig_num)
        sig_length = len(sig_data)
        #print("pywt.waverec 1")
        coeffs = pywt.wavedec(data=sig_data, wavelet='db8', level=9)
        cA9, cD9, cD8, cD7, cD6, cD5, cD4, cD3, cD2, cD1 = coeffs
        #print('here')
        # threshold
        threshold = (np.median(np.abs(cD1)) / 0.6745) * (np.sqrt(2 * np.log(len(cD1))))
        cD1.fill(0)
        cD2.fill(0)
        for i in range(1, len(coeffs) - 2):
           coeffs[i] = pywt.threshold(coeffs[i], threshold)
        #print("pywt.waverec 2")
        rdata = pywt.waverec(coeffs=coeffs, wavelet='db8')
        #print("denoise data:")
        sig_array2 = rdata
        zero_array = np.zeros(sig_length,)
        if max(sig_array2) != 0:
            #rdata = sig_array2[0:sig_length]
            for i in range(len(sig_array2)):
                #print("denoise loop")
                zero_array[i] = sig_array2[i]
            
            rdata = np.nan_to_num(zero_array)
        else:
            print("denoise zero error!")
            return sig_data
        
        #print("denoise return data:",rdata)
        return rdata

# def denoise_for_tf_data(data,label):
#   # Generating numpy array 
#   data_array = np.array(data).T
#   #print(data_array)
#   for i,sig_array  in enumerate(data_array):     
#     #print(type(sig_array))
#     # plt.figure()
#     # plt.plot(sig_array[0:1024]) 
#     # plt.show()
#     #print(type(sig_array))
#     sig_length = len(sig_array)
#     sig_array = denoise(sig_array)
#     #print(sig_array)
#     #plt.plot(sig_array[0:1024])
#     #plt.show()
#     data_array[i] = sig_array[0:sig_length].T
#   #print(data_array.shape)
#   data = tf.convert_to_tensor(data_array)
#   #print(data.numpy())
#   return data,label

def denoise_for_tf_data(data,label):
    try:
     # Generating numpy array 
      data_array = np.array(data).T
      #print(data_array)
      for i, sig_array in enumerate(data_array):     
        sig_length = len(sig_array)
        sig_array2 = denoise(sig_array,i)
        data_array[i] = sig_array2
      data2 = tf.convert_to_tensor(data_array.T)
#       if data_array.T.shape[0] != 5000 or data_array.T.shape[1] != 12:
# #       if data_array    
#           print("data_array shape:",data_array.T.shape)
#           print("data shape:",data2.shape)
      return data2,label        
    except:
      print('Denoise error!' )
      #print(data.numpy())
      return data,label

In [28]:
def get_dataset(df,batch_size, i):
    dataset = load_data(df, batch_size,i)
    ds_denoise = dataset.map(lambda x,y: 
                              tuple(tf.py_function(denoise_for_tf_data,[x,y],[tf.float16,tf.float32],)))
    ds_batch = ds_denoise.batch(128)
    return ds_batch

In [29]:
def show(data, label):
  for sig_data in data.T:
    plt.figure()
    plt.title(label)
    plt.plot(sig_data)
    plt.show()

In [30]:
#get test tensor dataset
test_dataset = get_dataset(test_df,len(test_df),0)
#test_dataset = test_dataset.prefetch(tf.data.experimental.AUTOTUNE)
#test_dataset = test_dataset.cache()
#test_dataset = test_dataset.batch(32)
test_dataset

<BatchDataset shapes: (<unknown>, <unknown>), types: (tf.float16, tf.float32)>

In [31]:
# test function
ds_test0 = load_data(train_df,512, 1)
def show_dataset(dataset):
  iterator = iter(dataset)
  raw_example = next(iterator)
  parsed_data = raw_example[0].numpy()
  label = raw_example[1].numpy()
  print('batch_data:',parsed_data, 'shape:', parsed_data.shape, '\n batcah_labels:',label, 'shape:', label.shape)
  print('batch_data:',parsed_data, 'shape:', parsed_data.shape, '\n batcah_labels:',label, 'shape:', label.shape)
show_dataset(ds_test0)

batch_data: [[-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 [-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 [-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 ...
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]] shape: (5000, 12) 
 batcah_labels: [1. 0.] shape: (2,)
batch_data: [[-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 [-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 [-0.085 -0.085  0.    ...  0.185  0.32   0.45 ]
 ...
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]
 [-0.08  -0.2   -0.12  ... -0.235 -0.22  -0.155]] shape: (5000, 12) 
 batcah_labels: [1. 0.] shape: (2,)


In [32]:
def show_dataset_bactched(dataset):
  iterator = iter(dataset)
  raw_example = next(iterator)
  parsed_data = raw_example[0].numpy()
  label = raw_example[1].numpy()
  #print('batch_data:',parsed_data, 'shape:', parsed_data.shape, '\n batcah_labels:',label, 'shape:', label.shape)
#   for i in range(len(label)):
#     show(parsed_data[0],label[0])  

In [32]:
# show_dataset_bactched(test_dataset)

# Modeling

In [33]:
# %tensorflow_version 2.x
# import tensorflow as tf
# print("Tensorflow version " + tf.__version__)

# try:
#   tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
#   print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
# except ValueError:
#   raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')

# tf.config.experimental_connect_to_cluster(tpu)
# tf.tpu.experimental.initialize_tpu_system(tpu)
# tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

In [34]:
# #CNN modeling
# def buildModel():

#     newModel = tf.keras.models.Sequential([
                                           
#         tf.keras.layers.InputLayer(input_shape=(12,5000)),

#         tf.keras.layers.Conv1D(filters=4, kernel_size=21, strides=1, padding='SAME', activation='relu'),
       
#         tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
        
#         tf.keras.layers.Conv1D(filters=16, kernel_size=23, strides=1, padding='SAME', activation='relu'),
        
#         tf.keras.layers.AvgPool1D(pool_size=1, strides=4, padding='SAME'),
       
#         tf.keras.layers.Conv1D(filters=32, kernel_size=25, strides=1, padding='SAME', activation='elu'),
       
#         tf.keras.layers.AvgPool1D(pool_size=1, strides=8, padding='SAME'),
       
#         tf.keras.layers.Conv1D(filters=64, kernel_size=27, strides=1, padding='SAME', activation='relu'),
       
#         tf.keras.layers.Flatten(),
       
#         tf.keras.layers.Dense(128, activation='relu'),
       
#         tf.keras.layers.Dropout(rate=0.2),
       
#         tf.keras.layers.Dense(2, activation='sigmoid')
#     ])

#     # newModel.compile(optimizer = 'adam' , 
#     #               loss = "categorical_crossentropy", metrics=["accuracy"])

#     return newModel

In [35]:
# # with tpu_strategy.scope():
# model_test = buildModel()
# model_test.summary()

# Initial optimizer and learning rate

In [5]:
initial_learning_rate = 0.1
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True) 
optimizer_RMS_schedule = tf.keras.optimizers.RMSprop(learning_rate=lr_schedule)
optimizer_adam = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False, name='Adam')
optimizer_adam_schedule = tf.keras.optimizers.Adam(learning_rate = lr_schedule)
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss",patience = 2)

# ANN Modeling

In [6]:
#ANN modeling
ann_model = tf.keras.models.Sequential()
ann_model.add(tf.keras.Input(shape=(5000,12)))
ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(50,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.Dense(25,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
#ann_model.add(tf.keras.layers.BatchNormalization())
ann_model.add(tf.keras.layers.GlobalAveragePooling1D())
ann_model.add(tf.keras.layers.Dense(12,activation = 'relu'))
ann_model.add(tf.keras.layers.Dropout(0.3))
ann_model.add(tf.keras.layers.Dense(2,activation = 'sigmoid'))

In [7]:
ann_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 5000, 12)          48        
_________________________________________________________________
dense (Dense)                (None, 5000, 50)          650       
_________________________________________________________________
dense_1 (Dense)              (None, 5000, 50)          2550      
_________________________________________________________________
dense_2 (Dense)              (None, 5000, 50)          2550      
_________________________________________________________________
dropout (Dropout)            (None, 5000, 50)          0         
_________________________________________________________________
dense_3 (Dense)              (None, 5000, 25)          1275      
_________________________________________________________________
dropout_1 (Dropout)          (None, 5000, 25)          0

In [8]:
ann_model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [9]:
def train_model(train_dataset,val_dataset,model,epochs,batch_size):
    s1 = datetime.datetime.now()
    train_prefetch = train_dataset.batch(batch_size).prefetch(tf.data.experimental.AUTOTUNE)
    train_cache = train_prefetch.cache()
    val_dataset = val_dataset.batch(batch_size).cache()
    model.fit(train_cache,epochs=epochs,validation_data = val_dataset,callbacks=[callback])#add validation dataset
    s2 = datetime.datetime.now()
    print(s2-s1)
    return model

In [10]:
model_annabell = train_model(train_dataset,val_dataset,ann_model,10,512)

Epoch 1/10
20/20 [==============================] - 497s 24s/step - loss: 0.7147 - accuracy: 0.5057 - Recall: 0.0981 - Precision: 0.5987 - AUC: 0.1663 - val_loss: 0.6899 - val_accuracy: 0.5711 - val_Recall: 0.5711 - val_Precision: 0.5711 - val_AUC: 0.5359
Epoch 2/10
 3/20 [===>..........................] - ETA: 6:04 - loss: 0.6555 - accuracy: 0.7767 - Recall: 0.5535 - Precision: 1.0000 - AUC: 0.0000e+00

KeyboardInterrupt: 

In [132]:
def train_model_ann(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    #for i in range(int(len(train_df)/batch_size)+1):
    for i in range(int(len(train_df)/batch_size)):
        print('batch:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,len(val_df),i)
        ann_model.fit(train_cache,epochs=epochs,validation_data = val_data,callbacks=[callback])#add validation dataset
    s2 = datetime.datetime.now()
    print(s2-s1)
    return ann_model

# import datetime
# def train_model_ann(train_df,batch_size , epochs, sig_num, model):
#     model.compile(optimizer = "adam", 
#                   loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy()])
#     s1 = datetime.datetime.now()
#     for e in range(epochs):
#         print('round:', e) 
#         for i in range(int(len(train_df)/batch_size)+1):            
#             ds_batch = get_dataset(train_df,batch_size,i,sig_num)
#             train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
#             train_cache = train_prefetch.cache()
#     #     val_data = get_dataset(val_df,int(batch_size*0.2),i)
#             ann_model.fit(train_cache)#,callbacks = [cp_callback])#validation_data = val_df,
#     s2 = datetime.datetime.now()
#     print(s2-s1)
#     return ann_model

In [133]:
model_lumda = train_model_ann(train_df, len(train_df), 50)

batch: 0
Epoch 1/50
148/148 [==============================] - 610s 4s/step - loss: 0.6440 - accuracy: 0.6302 - Recall: 0.6317 - Precision: 0.6297 - AUC: 0.6997 - val_loss: 0.5949 - val_accuracy: 0.6450 - val_Recall: 0.6385 - val_Precision: 0.6469 - val_AUC: 0.8267
Epoch 2/50
148/148 [==============================] - 205s 1s/step - loss: 0.5202 - accuracy: 0.7630 - Recall: 0.7707 - Precision: 0.7591 - AUC: 0.8228 - val_loss: 0.5108 - val_accuracy: 0.7457 - val_Recall: 0.7457 - val_Precision: 0.7457 - val_AUC: 0.8558
Epoch 3/50
148/148 [==============================] - 200s 1s/step - loss: 0.4820 - accuracy: 0.7880 - Recall: 0.7953 - Precision: 0.7839 - AUC: 0.8512 - val_loss: 0.4607 - val_accuracy: 0.7922 - val_Recall: 0.8023 - val_Precision: 0.7864 - val_AUC: 0.8690
Epoch 4/50
148/148 [==============================] - 201s 1s/step - loss: 0.4591 - accuracy: 0.7997 - Recall: 0.8071 - Precision: 0.7953 - AUC: 0.8649 - val_loss: 0.4448 - val_accuracy: 0.7842 - val_Recall: 0.7880 - val

In [134]:
model_lumda.evaluate(test_dataset)

19/19 [==============================] - 64s 3s/step - loss: 0.4266 - accuracy: 0.7974 - Recall: 0.7959 - Precision: 0.7983 - AUC: 0.8868


[0.42660829424858093,
 0.7973923683166504,
 0.7959437370300293,
 0.798256516456604,
 0.8868166208267212]

In [136]:
model_lumda.save("/Users/xianghuihuang/Desktop/model_2020/ann.h5")

# CNN

In [93]:
cnn_model = tf.keras.models.Sequential()
cnn_model.add(tf.keras.Input(shape=(5000,12)))
cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 21, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 23, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 25, padding='same',activation = 'relu'))
cnn_model.add(tf.keras.layers.AvgPool1D(pool_size=1,strides=4,padding='same'))

cnn_model.add(tf.keras.layers.Conv1D(filters = 4, kernel_size = 27, padding='same',activation = 'relu'))

cnn_model.add(tf.keras.layers.Flatten())

cnn_model.add(tf.keras.layers.Dense(128,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(tf.keras.layers.Dense(64,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))
cnn_model.add(tf.keras.layers.Dense(32,activation='relu'))
cnn_model.add(tf.keras.layers.Dropout(0.3))

cnn_model.add(tf.keras.layers.Dense(2,activation='sigmoid'))

In [94]:
cnn_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_12 (Conv1D)           (None, 5000, 4)           1012      
_________________________________________________________________
average_pooling1d (AveragePo (None, 1250, 4)           0         
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 1250, 4)           372       
_________________________________________________________________
average_pooling1d_1 (Average (None, 313, 4)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 313, 4)            404       
_________________________________________________________________
average_pooling1d_2 (Average (None, 79, 4)             0         
_________________________________________________________________
conv1d_15 (Conv1D)           (None, 79, 4)            

In [96]:
cnn_model.compile(optimizer = 'adam', 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [100]:
def train_model_cnn(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,len(val_df),i)
        cnn_model.fit(train_cache,epochs=epochs,validation_data = val_data,callbacks = [callback])#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return cnn_model

In [101]:
model_cellur = train_model_cnn(train_df, len(train_df), 50)

round: 0
Epoch 1/50
148/148 [==============================] - 515s 3s/step - loss: 0.6032 - accuracy: 0.6331 - Recall: 0.6160 - Precision: 0.6348 - AUC: 0.7236 - val_loss: 0.4402 - val_accuracy: 0.8056 - val_Recall: 0.8073 - val_Precision: 0.8046 - val_AUC: 0.8780
Epoch 2/50
148/148 [==============================] - 86s 581ms/step - loss: 0.4122 - accuracy: 0.8210 - Recall: 0.8203 - Precision: 0.8215 - AUC: 0.8903 - val_loss: 0.4184 - val_accuracy: 0.8058 - val_Recall: 0.8065 - val_Precision: 0.8054 - val_AUC: 0.8957
Epoch 3/50
148/148 [==============================] - 85s 574ms/step - loss: 0.3772 - accuracy: 0.8415 - Recall: 0.8428 - Precision: 0.8406 - AUC: 0.9102 - val_loss: 0.4044 - val_accuracy: 0.8182 - val_Recall: 0.8211 - val_Precision: 0.8163 - val_AUC: 0.8958
Epoch 4/50
148/148 [==============================] - 85s 576ms/step - loss: 0.3576 - accuracy: 0.8525 - Recall: 0.8540 - Precision: 0.8514 - AUC: 0.9184 - val_loss: 0.3916 - val_accuracy: 0.8205 - val_Recall: 0.8211

In [102]:
model_cellur.evaluate(test_dataset)

19/19 [==============================] - 59s 3s/step - loss: 0.4105 - accuracy: 0.8125 - Recall: 0.8129 - Precision: 0.8122 - AUC: 0.8977


[0.41046783328056335,
 0.8125,
 0.8129138946533203,
 0.8122414946556091,
 0.8977363705635071]

In [103]:
model_cellur.save("/Users/xianghuihuang/Desktop/model_2020/cnn.h5")

# LeNet_5 model

In [81]:
#leNet_5 modeling
lenet_5_model = tf.keras.models.Sequential()
lenet_5_model.add(tf.keras.Input(shape=(5000,12)))
lenet_5_model.add(tf.keras.layers.Conv1D(filters = 6, kernel_size = 3, padding='same'))
lenet_5_model.add(tf.keras.layers.BatchNormalization())
lenet_5_model.add(tf.keras.layers.Activation('relu'))
lenet_5_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))

lenet_5_model.add(tf.keras.layers.Conv1D(filters = 16, kernel_size = 5,strides =1))
lenet_5_model.add(tf.keras.layers.BatchNormalization())
lenet_5_model.add(tf.keras.layers.Activation('relu'))
lenet_5_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))

lenet_5_model.add(tf.keras.layers.GlobalAveragePooling1D())

lenet_5_model.add(tf.keras.layers.Dense(64,activation = 'relu'))

lenet_5_model.add(tf.keras.layers.Dense(32,activation = 'relu'))

lenet_5_model.add(tf.keras.layers.Dropout(0.3))

lenet_5_model.add(tf.keras.layers.Dense(16,activation = 'relu'))

lenet_5_model.add(tf.keras.layers.Dropout(0.3))

lenet_5_model.add(tf.keras.layers.Dense(2,activation = 'softmax'))

In [82]:
lenet_5_model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_10 (Conv1D)           (None, 5000, 6)           222       
_________________________________________________________________
batch_normalization_10 (Batc (None, 5000, 6)           24        
_________________________________________________________________
activation_10 (Activation)   (None, 5000, 6)           0         
_________________________________________________________________
max_pooling1d_10 (MaxPooling (None, 2500, 6)           0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 2496, 16)          496       
_________________________________________________________________
batch_normalization_11 (Batc (None, 2496, 16)          64        
_________________________________________________________________
activation_11 (Activation)   (None, 2496, 16)         

In [83]:
lenet_5_model.compile(optimizer = "adam", 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [99]:
# checkpoint_path = '/Users/xianghuihuang/Desktop/model_2020/tf1/'
# checkpoint_dir = os.path.dirname(checkpoint_path)

# # Create a callback that saves the model's weights
# cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
#                                                  save_weights_only=True,
#                                                  verbose=1)

def train_model_lenet(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,len(val_df),i)
        lenet_5_model.fit(train_cache,epochs=epochs,validation_data = val_data,callbacks = [callback])#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return lenet_5_model

In [85]:
# train_df.iloc[1340].new_filename

In [86]:
# data = wfdb.rdrecord('/Users/xianghuihuang/Desktop/WFDB/HR19299').p_signal

In [87]:
# show(data,"debug")

In [88]:
# data2 = data.T
# for i,signal_data in enumerate(data2):
#     #print(signal_data.shape)
#     signal_data = denoise(signal_data,i)
#     data2[i] = signal_data
#     print(data2[i].shape)
# data3 = data2.T

In [89]:
# data2[11]

In [90]:
# show(data3,"after denoise debug")

In [91]:
#1024+256, 4096+512*24+128*3
model_alexa = train_model_lenet(train_df, len(train_df), 50)

round: 0
Epoch 1/50
148/148 [==============================] - 539s 4s/step - loss: 0.6539 - accuracy: 0.6235 - Recall: 0.6235 - Precision: 0.6235 - AUC: 0.6745 - val_loss: 0.5844 - val_accuracy: 0.7495 - val_Recall: 0.7495 - val_Precision: 0.7495 - val_AUC: 0.8283
Epoch 2/50
148/148 [==============================] - 93s 634ms/step - loss: 0.5200 - accuracy: 0.7670 - Recall: 0.7670 - Precision: 0.7670 - AUC: 0.8281 - val_loss: 0.4865 - val_accuracy: 0.7713 - val_Recall: 0.7713 - val_Precision: 0.7713 - val_AUC: 0.8564
Epoch 3/50
148/148 [==============================] - 91s 618ms/step - loss: 0.4625 - accuracy: 0.7940 - Recall: 0.7940 - Precision: 0.7940 - AUC: 0.8634 - val_loss: 0.4400 - val_accuracy: 0.7918 - val_Recall: 0.7918 - val_Precision: 0.7918 - val_AUC: 0.8808
Epoch 4/50
148/148 [==============================] - 92s 623ms/step - loss: 0.4313 - accuracy: 0.8111 - Recall: 0.8111 - Precision: 0.8111 - AUC: 0.8817 - val_loss: 0.4224 - val_accuracy: 0.8027 - val_Recall: 0.8027

In [92]:
model_alexa.evaluate(test_dataset)

19/19 [==============================] - 59s 3s/step - loss: 0.4355 - accuracy: 0.8075 - Recall: 0.8075 - Precision: 0.8075 - AUC: 0.8962


[0.43551161885261536,
 0.8075330853462219,
 0.8075330853462219,
 0.8075330853462219,
 0.8961711525917053]

In [104]:
model_alexa.save("/Users/xianghuihuang/Desktop/model_2020/leNet_5.h5")

# VGG16 model

In [1]:
#leNet_5 modeling
vgg16_model = tf.keras.models.Sequential()
vgg16_model.add(tf.keras.Input(shape=(5000,12)))
#64
vgg16_model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))
#128
vgg16_model.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 128, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))
#256
vgg16_model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 256, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))
#512
vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))

vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.Conv1D(filters = 512, kernel_size = 3, padding='same'))
vgg16_model.add(tf.keras.layers.BatchNormalization())
vgg16_model.add(tf.keras.layers.Activation('relu'))
vgg16_model.add(tf.keras.layers.MaxPool1D(pool_size=2,strides=2,padding='same'))

vgg16_model.add(tf.keras.layers.GlobalAveragePooling1D())

vgg16_model.add(tf.keras.layers.Dense(256,activation = 'relu'))
vgg16_model.add(tf.keras.layers.Dropout(0.5))
vgg16_model.add(tf.keras.layers.Dense(128,activation = 'relu'))
vgg16_model.add(tf.keras.layers.Dropout(0.5))
vgg16_model.add(tf.keras.layers.Dense(64,activation = 'relu'))
vgg16_model.add(tf.keras.layers.Dropout(0.5))
vgg16_model.add(tf.keras.layers.Dense(32,activation = 'relu'))
vgg16_model.add(tf.keras.layers.Dropout(0.5))

vgg16_model.add(tf.keras.layers.Dense(2,activation = 'softmax'))

NameError: name 'tf' is not defined

In [2]:
vgg16_model.summary()

NameError: name 'vgg16_model' is not defined

In [52]:
vgg16_model.compile(optimizer = "adam", 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [53]:
def train_model_vgg16(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        #val_data = get_dataset(val_df,len(val_df),i)
        vgg16_model.fit(train_cache,epochs=epochs)#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return vgg16_model

In [54]:
model_venessa = train_model_vgg16(train_df, len(train_df), 10)

round: 0
Epoch 1/10
 34/148 [=====>........................] - ETA: 27:35 - loss: 0.7615 - accuracy: 0.5176 - Recall: 0.5176 - Precision: 0.5176 - AUC: 0.5331

/Users/xianghuihuang/anaconda3/lib/python3.7/site-packages/pywt/_thresholding.py:23: RuntimeWarning: invalid value encountered in true_divide
  thresholded = (1 - value/magnitude)


148/148 [==============================] - 2162s 15s/step - loss: 0.6675 - accuracy: 0.6003 - Recall: 0.6003 - Precision: 0.6003 - AUC: 0.6467
Epoch 2/10
148/148 [==============================] - 2079s 14s/step - loss: 0.4212 - accuracy: 0.8265 - Recall: 0.8265 - Precision: 0.8265 - AUC: 0.8951
Epoch 3/10
148/148 [==============================] - 2062s 14s/step - loss: 0.3796 - accuracy: 0.8498 - Recall: 0.8498 - Precision: 0.8498 - AUC: 0.9151
Epoch 4/10
148/148 [==============================] - 2052s 14s/step - loss: 0.3575 - accuracy: 0.8552 - Recall: 0.8552 - Precision: 0.8552 - AUC: 0.9228
Epoch 5/10
148/148 [==============================] - 2053s 14s/step - loss: 0.3459 - accuracy: 0.8589 - Recall: 0.8589 - Precision: 0.8589 - AUC: 0.9267
Epoch 6/10
148/148 [==============================] - 2046s 14s/step - loss: 0.3290 - accuracy: 0.8635 - Recall: 0.8635 - Precision: 0.8635 - AUC: 0.9334
Epoch 7/10
148/148 [==============================] - 2045s 14s/step - loss: 0.3266 - a

In [ ]:
model_venessa.evaluate(test_dataset)

In [56]:
#save your best model 
model_venessa.save("/Users/xianghuihuang/Desktop/model_2020/vgg16.h5")

# Inception

In [43]:
def inception_block(prev_layer):
    
    conv1=tf.keras.layers.Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv1=tf.keras.layers.BatchNormalization()(conv1)
    conv1=tf.keras.layers.Activation('relu')(conv1)
    
    conv3=tf.keras.layers.Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv3=tf.keras.layers.BatchNormalization()(conv3)
    conv3=tf.keras.layers.Activation('relu')(conv3)
    conv3=tf.keras.layers.Conv1D(filters = 64, kernel_size = 3, padding = 'same')(conv3)
    conv3=tf.keras.layers.BatchNormalization()(conv3)
    conv3=tf.keras.layers.Activation('relu')(conv3)
    
    conv5=tf.keras.layers.Conv1D(filters = 64, kernel_size = 1, padding = 'same')(prev_layer)
    conv5=tf.keras.layers.BatchNormalization()(conv5)
    conv5=tf.keras.layers.Activation('relu')(conv5)
    conv5=tf.keras.layers.Conv1D(filters = 64, kernel_size = 5, padding = 'same')(conv5)
    conv5=tf.keras.layers.BatchNormalization()(conv5)
    conv5=tf.keras.layers.Activation('relu')(conv5)
    
    pool= tf.keras.layers.MaxPool1D(pool_size=3, strides=1, padding='same')(prev_layer)
    convmax=tf.keras.layers.Conv1D(filters = 64, kernel_size = 1, padding = 'same')(pool)
    convmax=tf.keras.layers.BatchNormalization()(convmax)
    convmax=tf.keras.layers.Activation('relu')(convmax)
    
    layer_out = tf.keras.layers.concatenate([conv1, conv3, conv5, convmax])
    
    return layer_out

def inception_model(input_shape):
    X_input=tf.keras.layers.Input(input_shape)
    
    X = tf.keras.layers.ZeroPadding1D(3)(X_input)
    
    X = tf.keras.layers.Conv1D(filters = 64, kernel_size = 7, padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    X = tf.keras.layers.MaxPool1D(pool_size=3, strides=2, padding='same')(X)
    
    X = tf.keras.layers.Conv1D(filters = 64, kernel_size = 1, padding = 'same')(X)
    X = tf.keras.layers.BatchNormalization()(X)
    X = tf.keras.layers.Activation('relu')(X)
    
    X = inception_block(X)
    X = inception_block(X)
    
    X = tf.keras.layers.MaxPool1D(pool_size=7, strides=2, padding='same')(X)
    
    X = tf.keras.layers.GlobalAveragePooling1D()(X)
    X = tf.keras.layers.Dense(32,activation='relu')(X)
    
    X = tf.keras.layers.Dense(2,activation='sigmoid')(X)
    
    model = tf.keras.Model(inputs = X_input, outputs = X, name='Inception')
    
    return model

In [44]:
inception_model = inception_model(input_shape = (5000,12))

In [45]:
inception_model.summary()

Model: "Inception"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 5000, 12)]   0                                            
__________________________________________________________________________________________________
zero_padding1d_1 (ZeroPadding1D (None, 5006, 12)     0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_14 (Conv1D)              (None, 5006, 64)     5440        zero_padding1d_1[0][0]           
__________________________________________________________________________________________________
batch_normalization_15 (BatchNo (None, 5006, 64)     256         conv1d_14[0][0]                  
__________________________________________________________________________________________

In [46]:
inception_model.compile(optimizer = "adam", 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [47]:
def train_model_inception(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)+1):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,int(batch_size*0.2),i)
        inception_model.fit(train_cache,epochs=epochs,validation_data = val_data)#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return inception_model

In [48]:
model_mars = train_model_inception(train_df, 2048, 20)

round: 0
Epoch 1/20
16/16 [==============================] - 143s 9s/step - loss: 0.5790 - accuracy: 0.7017 - Recall: 0.7995 - Precision: 0.6821 - AUC: 0.7688 - val_loss: 0.6675 - val_accuracy: 0.7518 - val_Recall: 0.6553 - val_Precision: 0.8121 - val_AUC: 0.8467
Epoch 2/20
16/16 [==============================] - 119s 7s/step - loss: 0.4188 - accuracy: 0.8117 - Recall: 0.8124 - Precision: 0.8112 - AUC: 0.8896 - val_loss: 0.6405 - val_accuracy: 0.7641 - val_Recall: 0.7237 - val_Precision: 0.7872 - val_AUC: 0.8375
Epoch 3/20
16/16 [==============================] - 119s 7s/step - loss: 0.3750 - accuracy: 0.8292 - Recall: 0.8283 - Precision: 0.8298 - AUC: 0.9119 - val_loss: 0.6272 - val_accuracy: 0.7152 - val_Recall: 0.6846 - val_Precision: 0.7292 - val_AUC: 0.8192
Epoch 4/20
16/16 [==============================] - 120s 8s/step - loss: 0.3488 - accuracy: 0.8351 - Recall: 0.8340 - Precision: 0.8358 - AUC: 0.9253 - val_loss: 0.6231 - val_accuracy: 0.6883 - val_Recall: 0.6699 - val_Precisi

In [49]:
model_mars.evaluate(test_dataset)

19/19 [==============================] - 74s 4s/step - loss: 0.9012 - accuracy: 0.7873 - Recall: 0.7850 - Precision: 0.7886 - AUC: 0.8565


[0.9012466669082642,
 0.7873009443283081,
 0.7849957942962646,
 0.7886315584182739,
 0.8564768433570862]

In [50]:
#save inception model here
model_mars.save("/Users/xianghuihuang/Desktop/model_2020/inception.h5")

# LSTM Modeling

In [107]:
lstm_model = tf.keras.models.Sequential()
lstm_model.add(tf.keras.layers.LSTM(64, input_shape=(5000,12), return_sequences=True))
lstm_model.add(tf.keras.layers.BatchNormalization())
lstm_model.add(tf.keras.layers.LSTM(64))
lstm_model.add(tf.keras.layers.Dense(32, activation = 'relu'))
lstm_model.add(tf.keras.layers.Dropout(0.3))
lstm_model.add(tf.keras.layers.Dense(16, activation = 'relu'))
lstm_model.add(tf.keras.layers.Dropout(0.3))
lstm_model.add(tf.keras.layers.Dense(2, activation = 'sigmoid'))

In [108]:
lstm_model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 5000, 64)          19712     
_________________________________________________________________
batch_normalization_12 (Batc (None, 5000, 64)          256       
_________________________________________________________________
lstm_1 (LSTM)                (None, 64)                33024     
_________________________________________________________________
dense_28 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_9 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_10 (Dropout)         (None, 16)               

In [109]:
lstm_model.compile(optimizer = "adam", 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [110]:
def train_model_lstm(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,len(val_df),i)
        lstm_model.fit(train_cache,epochs=epochs,validation_data = val_data,callbacks = [callback])#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return lstm_model

In [112]:
model_lunar = train_model_lstm(train_df, len(train_df), 1)

round: 0
148/148 [==============================] - 1205s 8s/step - loss: 0.6357 - accuracy: 0.6310 - Recall: 0.6195 - Precision: 0.6341 - AUC: 0.6799 - val_loss: 0.6422 - val_accuracy: 0.6125 - val_Recall: 0.6041 - val_Precision: 0.6144 - val_AUC: 0.6691
0:21:47.309905


In [113]:
model_lunar.evaluate(test_dataset)

19/19 [==============================] - 69s 4s/step - loss: 0.6381 - accuracy: 0.6322 - Recall: 0.6262 - Precision: 0.6339 - AUC: 0.6825


[0.6381473541259766,
 0.6322433948516846,
 0.6262417435646057,
 0.6338500380516052,
 0.682476282119751]

In [114]:
#save LSTM model here
model_lunar.save("/Users/xianghuihuang/Desktop/model_2020/lstm.h5")

# LSTM+CNN Modeling

In [120]:
time_step = 5000
num_sensors = 12

def conv_block_type1(x,num_filters, kernel_size):
    x = tf.keras.layers.Conv1D(num_filters, kernel_size, activation='relu', padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv1D(num_filters, kernel_size, activation='relu', padding="same")(x)
    x = tf.keras.layers.MaxPooling1D()(x)

    return x


def conv_block_type2(x,num_filters, kernel_size):
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv1D(num_filters, kernel_size, activation='relu', padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Conv1D(num_filters, kernel_size, activation='relu', padding="same")(x)
    x = tf.keras.layers.MaxPooling1D()(x)

    return x


def build_model(time_step,num_sensors):
    input_shape = (time_step,num_sensors)

    #CNNModel
    inpt = tf.keras.layers.Input(input_shape)

    x = tf.keras.layers.Conv1D(32, 16, activation='relu')(inpt)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)

    x = conv_block_type1(x, 32, 16)

    x = conv_block_type2(x, 32, 16)
    x = conv_block_type2(x, 64, 8)
    x = conv_block_type2(x, 64, 8)
    x = conv_block_type2(x, 128, 4)
    x = conv_block_type2(x, 128, 4)

    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.ReLU()(x)
    x = tf.keras.layers.Bidirectional(LSTM(64, return_sequences=True, input_shape=(None, 1)))(x)
    x = tf.keras.layers.GlobalMaxPooling1D()(x)

    x = tf.keras.layers.Dense(32, activation='relu', input_shape=(None, 128))(x)
    x = tf.keras.layers.Dense(2, activation='sigmoid')(x)

    model = tf.keras.Model(inputs=inpt, outputs=x)

    return(model)

In [121]:
cnn_lstm_model = build_model(time_step,num_sensors)

In [122]:
cnn_lstm_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         [(None, 5000, 12)]        0         
_________________________________________________________________
conv1d_29 (Conv1D)           (None, 4985, 32)          6176      
_________________________________________________________________
batch_normalization_26 (Batc (None, 4985, 32)          128       
_________________________________________________________________
re_lu_13 (ReLU)              (None, 4985, 32)          0         
_________________________________________________________________
conv1d_30 (Conv1D)           (None, 4985, 32)          16416     
_________________________________________________________________
batch_normalization_27 (Batc (None, 4985, 32)          128       
_________________________________________________________________
re_lu_14 (ReLU)              (None, 4985, 32)          0   

In [123]:
cnn_lstm_model.compile(optimizer = "adam", 
                  loss = tf.keras.losses.BinaryCrossentropy(), metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy', dtype=None, threshold=0.5),tf.keras.metrics.Recall(name='Recall'),tf.keras.metrics.Precision(name='Precision'), 
                    tf.keras.metrics.AUC(
        num_thresholds=200,
        curve="ROC",
        summation_method="interpolation",
        name="AUC",
        dtype=None,
        thresholds=None,
        multi_label=True,
        label_weights=None,)])

In [125]:
def train_model_cnn_lstm(train_df,batch_size , epochs):
    s1 = datetime.datetime.now()
    for i in range(int(len(train_df)/batch_size)):
        print('round:', i)  
        ds_batch = get_dataset(train_df,batch_size,i)
        train_prefetch = ds_batch.prefetch(tf.data.experimental.AUTOTUNE)
        train_cache = train_prefetch.cache()
        val_data = get_dataset(val_df,len(val_df),i)
        cnn_lstm_model.fit(train_cache,epochs=epochs,validation_data = val_data,callbacks=[callback])#,callbacks = [cp_callback])#validation_data = val_df,
    s2 = datetime.datetime.now()
    print(s2-s1)
    return cnn_lstm_model

In [126]:
model_bella = train_model_cnn_lstm(train_df, len(train_df), 20)

round: 0
Epoch 1/20
148/148 [==============================] - 946s 6s/step - loss: 0.6308 - accuracy: 0.6218 - Recall: 0.6191 - Precision: 0.6246 - AUC: 0.6689 - val_loss: 0.7254 - val_accuracy: 0.5023 - val_Recall: 0.5023 - val_Precision: 0.5023 - val_AUC: 0.5000
Epoch 2/20
148/148 [==============================] - 757s 5s/step - loss: 0.4421 - accuracy: 0.7942 - Recall: 0.7923 - Precision: 0.7952 - AUC: 0.8724 - val_loss: 0.6937 - val_accuracy: 0.5027 - val_Recall: 0.5027 - val_Precision: 0.5027 - val_AUC: 0.4348
Epoch 3/20
148/148 [==============================] - 730s 5s/step - loss: 0.3984 - accuracy: 0.8193 - Recall: 0.8179 - Precision: 0.8202 - AUC: 0.8974 - val_loss: 0.7231 - val_accuracy: 0.3473 - val_Recall: 0.3159 - val_Precision: 0.3371 - val_AUC: 0.2368
Epoch 4/20
148/148 [==============================] - 729s 5s/step - loss: 0.3860 - accuracy: 0.8291 - Recall: 0.8269 - Precision: 0.8305 - AUC: 0.9031 - val_loss: 0.7295 - val_accuracy: 0.3620 - val_Recall: 0.3431 - val

In [127]:
model_bella.evaluate(test_dataset)

19/19 [==============================] - 68s 4s/step - loss: 0.7270 - accuracy: 0.3789 - Recall: 0.3572 - Precision: 0.3734 - AUC: 0.3029


[0.7270005345344543,
 0.3789321184158325,
 0.3572019934654236,
 0.37343141436576843,
 0.302914023399353]

In [128]:
model_bella.save("/Users/xianghuihuang/Desktop/model_2020/cnn_lstm.h5")

In [ ]:
#save CNN_LSTM model here

In [ ]:
# inceptionV3_model = tf.keras.applications.InceptionV3(input_shape = (5000,12),include_top= False)